In [1]:

%matplotlib inline
import numpy as np
import pandas as pd
import datetime  
import warnings
import missingno as msno

In [2]:
path_train = "./data/cs-training.csv"  # 训练文件
path_test = "./data/cs-test.csv"  # 测试文件

df = pd.read_csv(path_train,index_col='Unnamed: 0')

In [ ]:
# msno.matrix(df)

In [ ]:
# 均值填充
for i in df.columns:
    df[i].fillna(np.mean(df[i]),inplace=True)

# df[i].fillna(np.mean(df[i]),inplace=True)

# 盖帽法
for i in df.columns:
     df[i] = cap(df[i],quantile=[0.05,0.95])

# df[i] = cap(df[i],quantile=[0.05,0.95])

df['sample_weight'] = df['SeriousDlqin2yrs'].map(lambda x:0.8 if x==1 else 0.2)
data_train = df
from sklearn.model_selection import train_test_split
X = data_train.iloc[:, 1:-1]
y = data_train.iloc[:, 0]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=8)

# df_train = pd.concat([y_train, X_train], axis=1)
# df_val = pd.concat([y_val, X_val], axis=1)

In [149]:
# msno.matrix(df)

In [258]:
# df['SeriousDlqin2yrs']

In [263]:
# # 设置样本权重
# data['temp_label'] = data['score']
# # 这里设置为None 而不是删除该数据，因为删除的话，线下一定是提升的，对于线上而言，异常数据依旧存在，所以应该关注在训练集无异常，而测试集有异常下的处理效果
# data['sample_weight'] = data['temp_label'] + 200
# data['sample_weight'] = data['sample_weight'] / data['sample_weight'].mean()
# # 方案1 ，不训练
# data.loc[data.id.isin(ab_id), 'temp_label'] = None
# # 方案2，样本权重设置低一点
# data.loc[data.id.isin(ab_id), 'sample_weight'] = 0.01


案例

In [3]:
def cap(x,quantile=[0.01,0.99]):
    """盖帽法处理异常值
    Args：
        x：pd.Series列，连续变量
        quantile：指定盖帽法的上下分位数范围
    """

# 生成分位数
    Q01,Q99=x.quantile(quantile).values.tolist()

# 替换异常值为指定的分位数
    if Q01 > x.min():
        x = x.copy()
        x.loc[x<Q01] = Q01

    if Q99 < x.max():
        x = x.copy()
        x.loc[x>Q99] = Q99

    return(x)

In [4]:
def prod_trian_data(path_train):
    """
    处理训练数据集
    :return: 
    """
    df = pd.read_csv(path_train,index_col='Unnamed: 0')


    # 均值填充
    for i in df.columns:
        df[i].fillna(np.mean(df[i]),inplace=True)

    # df[i].fillna(np.mean(df[i]),inplace=True)

    # 盖帽法
    for i in df.columns:
         df[i] = cap(df[i],quantile=[0.05,0.95])

    # df[i] = cap(df[i],quantile=[0.05,0.95])

    data_train = df
    from sklearn.model_selection import train_test_split
    X = data_train.iloc[:, 1:]
    y = data_train.iloc[:, 0]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=8)

#     df_train = pd.concat([y_train, X_train], axis=1)
#     df_val = pd.concat([y_val, X_val], axis=1)
    return X_train, X_val, y_train, y_val,data_train

In [5]:
def prod_test_data(path_test):

    """
    处理测试集
    :return: 
    """
    df = pd.read_csv(path_test,index_col='Unnamed: 0')    
    
    # 均值填充
    for i in df.columns:
        df[i].fillna(np.mean(df[i]),inplace=True)

    # df[i].fillna(np.mean(df[i]),inplace=True)

    # 盖帽法
    for i in df.columns:
         df[i] = cap(df[i],quantile=[0.05,0.95])

    # df[i] = cap(df[i],quantile=[0.05,0.95])


    data_test = df    
    X_test = data_test.iloc[:, 1:]
    X_test_df_index = data_test.index
    
    return X_test,X_test_df_index,data_test 

In [6]:
def predict_model():
    # 模型1:复赛0.08成绩代码改为全样本
    from xgboost import XGBClassifier
    xgbr = XGBClassifier()
    xgbr.fit(X_train, y_train)
    
    
#     from lightgbm import LGBMClassifier
#     lgbr = LGBMClassifier() 
# #     lgbr1 = LGBMClassifier(learning_rate=0.01,reg_alpha=1,max_depth=6,min_child_weight=4) 
#     lgbr.fit(X_train, y_train)

#     from catboost import CatBoostClassifier
#     cgbr = CatBoostClassifier()
#     cgbr = CatBoostRClassifier(iterations=250,depth=5,learning_rate=0.01,l2_leaf_reg=1)
#     iterations=2, depth=2, learning_rate=1, loss_function='RMSE'
#     cgbr.fit(X_train, y_train)

# 模型评价
    from sklearn.metrics import accuracy_score
    from sklearn import metrics
    from sklearn.metrics import confusion_matrix 
    y_pred = xgbr.predict(X_val)
    print(metrics.f1_score(y_val, y_pred)) 

# 模型预测结果
    Id_pred=pd.DataFrame()
    Id_pred['CustomerID'] = X_test_df_index

    Id_pred['pred_xgbr']=xgbr.predict(X_test)
#     Id_pred['pred_lgbr']=lgbr.predict(X_test)
    # Id_pred['pred_cgbr']=cgbr.predict(X_test)

    
# 结果融合
    Id_pred['SeriousDlqin2yrs'] = Id_pred['pred_xgbr']
    del Id_pred['pred_xgbr']
#     Id_pred['SeriousDlqin2yrs']= 0.5*Id_pred['pred_xgbr']+ 0.5*Id_pred['pred_xgbr']
#     del Id_pred['pred_xgbr'],Id_pred['pred_lgbr']
    Id_pred.to_csv("./submit/张志祥_results_b.csv",index=None)

In [7]:
# if __name__ == "__main__":
#     old_time=datetime.datetime.now()
    
#     path_train = "./data/cs-training.csv"  # 训练文件
#     path_test = "./data/cs-test.csv"  # 测试文件
    
#     print("****************** start **********************")

#     X_train, X_val, y_train, y_val = prod_trian_data(path_train)#     训练集数据
#     X_test,X_test_df_index = prod_test_data(path_test) #  测试集数据
    
#     predict_model()#模型训练输出结果
    
#     now_time=datetime.datetime.now()
#     print("runtime",(now_time-old_time).seconds,"s")#输出模型训练用时

In [8]:
X_train, X_val, y_train, y_val,data_train  = prod_trian_data(path_train)#     训练集数据
X_test,X_test_df_index,data_test  = prod_test_data(path_test) #  测试集数据

描述性统计

In [9]:
def get_df_train_describe(df,col_name):
#     col_name = '训练集'
    df_describe = df.describe().iloc[[1,3,4,5,6,7],1:]
    df_describe.index = ['Mean ', 'Min.', '1st Qu.', 'Median', '3rd Qu.','Max. ']
    df_describe_T = df_describe.loc[['Min.', '1st Qu.', 'Median', 'Mean ', '3rd Qu.','Max. '],:].T
    df_describe_T['NA counts'] = df.iloc[:,1:].apply(lambda x:x.isnull().sum())

    df_describe = df_describe_T.T
    df_describe = df_describe.reset_index()
    df_describe = df_describe.rename(columns={'index':col_name})
    return df_describe

In [10]:
# get_df_train_describe(data_train,'训练集')#.to_excel("./submit/张志祥_results_a_train.xlsx",index=None)

In [11]:
# get_df_train_describe(data_test,"测试集")#.to_excel("./submit/张志祥_results_a_test.xlsx",index=None)

In [12]:
# X_train['sample_weight'] = pd.DataFrame(y_train)['SeriousDlqin2yrs'].map(lambda x:0.8 if x==1 else 0.2)

In [13]:
# X_train['sample_weight']

In [14]:
# XGBClassifier?

In [15]:
df['SeriousDlqin2yrs'].value_counts()

0    139974
1     10026
Name: SeriousDlqin2yrs, dtype: int64

In [39]:
# 模型
from xgboost import XGBClassifier
# class_weight = dict({0:14, 1:1})
# sample_weight = pd.DataFrame(y_train)['SeriousDlqin2yrs'].map(lambda x:0.99 if x==1 else 0.01)
# alpha
#     L1正则化系数，默认为1
# lambda


xgbr = XGBClassifier(scale_pos_weight=4)
# xgbr = XGBClassifier()
xgbr.fit(X_train, y_train)
y_pred = xgbr.predict(X_val)
print(metrics.f1_score(y_val.values, y_pred)) 

0.44002921840759673


D:\Users\DELL\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [36]:
# for i in range(6,1,-1):
#     print(i)
#     xgbr = XGBClassifier(scale_pos_weight=i)
#     # xgbr = XGBClassifier()
#     xgbr.fit(X_train, y_train)
#     y_pred = xgbr.predict(X_val)
#     print(metrics.f1_score(y_val.values, y_pred)) 

In [23]:
# from lightgbm import LGBMClassifier
# lgbr = LGBMClassifier() 
# #     lgbr1 = LGBMClassifier(learning_rate=0.01,reg_alpha=1,max_depth=6,min_child_weight=4) 
# lgbr.fit(X_train, y_train)

In [24]:
# from catboost import CatBoostClassifier
# cgbr = CatBoostClassifier(iterations=250,depth=5,learning_rate=0.01,l2_leaf_reg=1)
# cgbr.fit(X_train, y_train)

In [40]:
# 模型评价
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix 

In [41]:
y_pred = xgbr.predict(X_val)
print(metrics.f1_score(y_val.values, y_pred)) 
print(classification_report(y_val.values, y_pred))

0.44002921840759673
             precision    recall  f1-score   support

          0       0.96      0.94      0.95     41974
          1       0.39      0.50      0.44      3026

avg / total       0.92      0.91      0.92     45000



D:\Users\DELL\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [21]:
y_pred = xgbr.predict(X_val)
print(metrics.f1_score(y_val.values, y_pred)) 
print(classification_report(y_val.values, y_pred))

0.38064688585939593
             precision    recall  f1-score   support

          0       0.98      0.86      0.91     41974
          1       0.26      0.71      0.38      3026

avg / total       0.93      0.85      0.88     45000



D:\Users\DELL\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [293]:
# 模型预测结果融合
Id_pred=pd.DataFrame()
Id_pred['pred_xgbr_proba'] = xgbr.predict_proba(X_val)[:, 1]
Id_pred['pred_xgbr'] = Id_pred['pred_xgbr_proba'].map(lambda x:1 if x>0.5 else 0)

y_pred = Id_pred['pred_xgbr']

In [220]:
# F-measure的含义是：用precision和recall两个指标不直观（任性），
# 索性把他们合并为一个，这就是F-measure（越大越好，1为理想状态，此时precision为1，recall为1）

print(metrics.f1_score(y_val.values, y_pred)) 
# print(accuracy_score(y_val.values, y_pred)) # ACC
print(metrics.precision_score(y_val.values, y_pred))  #  精确率
print(metrics.recall_score(y_val.values, y_pred))  #  精确率
# print(confusion_matrix(y_val.values, y_pred))

0.2582385816149547
0.5726495726495726
0.16670813635232645


In [ ]:
# 模型预测结果融合
Id_pred=pd.DataFrame()
Id_pred['CustomerID'] = X_test_df_index

Id_pred['pred_xgbr']=xgbr.predict(X_test)
# Id_pred['pred_lgbr']=lgbr.predict(X_test)
# Id_pred['pred_cgbr']=cgbr.predict(X_test)

Id_pred['SeriousDlqin2yrs'] = Id_pred['pred_xgbr']
del Id_pred['pred_xgbr']
# Id_pred['SeriousDlqin2yrs']= 0.5*Id_pred['pred_xgbr']+ 0.5*Id_pred['pred_xgbr']
# del Id_pred['pred_xgbr'],Id_pred['pred_lgbr']
Id_pred.to_csv("./submit/XXX_results_b.csv",index=None)